In [1]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text, desc
from sqlalchemy.sql import func, distinct, exists
from sqlalchemy import union, union_all

engine = create_engine('sqlite+pysqlite:////Users/vladislavvorosilov/Desktop/Python_SQL/Python_SQL_PI_2021_22_s2/Data/Students.db')
metadata_obj = MetaData()
metadata_obj.reflect(bind=engine) 
exam_marks=Table("exam_marks", metadata_obj, autoload=True, autoload_with="engine")
lecturer=Table("lecturer", metadata_obj, autoload=True, autoload_with="engine")
student=Table("student", metadata_obj, autoload=True, autoload_with="engine")
subj_lect=Table("subj_lect", metadata_obj, autoload=True, autoload_with="engine")
subject=Table("subject", metadata_obj, autoload=True, autoload_with="engine")
university=Table("university", metadata_obj, autolad=True, autolad_with="engine")
conn=engine.connect()


## 1. Студентам, обучающимся в ВГУ, присвойте порядковый номер, упорядочив по фамилии и имени.

In [7]:
subq = select(university.c.univ_id)
subq = subq.where(university.c.univ_name == "ВГУ").scalar_subquery()

stmt = select(func.row_number().over(order_by=(student.c.surname, student.c.name)), student)
stmt = stmt.select_from(student)
stmt = stmt.where(student.c.univ_id == subq)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result[:10]

SELECT row_number() OVER (ORDER BY student.surname, student.name) AS anon_1, student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student 
WHERE student.univ_id = (SELECT university.univ_id 
FROM university 
WHERE university.univ_name = :univ_name_1)
16


[(1, 78, 'Агафонов', 'Василий', 150, 1, 'Грозный', datetime.datetime(1994, 10, 29, 0, 0), 10),
 (2, 113, 'Аксенов', 'Валерий', 150, 1, 'Мурманск', datetime.datetime(1994, 10, 27, 0, 0), 10),
 (3, 55, 'Белкин', 'Вадим', 250, 5, 'Воронеж', datetime.datetime(2000, 1, 7, 0, 0), 10),
 (4, 99, 'Борисова', 'Марина', 150, 1, 'Рославль', datetime.datetime(1995, 8, 7, 0, 0), 10),
 (5, 88, 'Жданова', 'Ирина', 200, 3, 'Тверь', datetime.datetime(2002, 3, 2, 0, 0), 10),
 (6, 12, 'Зайцева', 'Ольга', 250, 2, 'Липецк', datetime.datetime(2001, 5, 1, 0, 0), 10),
 (7, 1, 'Иванов', 'Иван', 150, 1, 'Орел', datetime.datetime(2002, 12, 3, 0, 0), 10),
 (8, 10, 'Кузнецов', 'Борис', 0, 2, 'Брянск', datetime.datetime(2001, 12, 8, 0, 0), 10),
 (9, 654, 'Лукин', 'Артем', 200, 3, 'Воронеж', datetime.datetime(2001, 12, 1, 0, 0), 10),
 (10, 76, 'Мельникова', 'Дарья', 100, 5, 'Архангельск', datetime.datetime(2002, 8, 1, 0, 0), 10)]

## 2. Предметы, по которым сдавались экзамены, ранжируйте по среднему баллу, полученному на экзаменах. Результат должен содержать название предмета, средний балл, номер ранга без пропусков. Предметам с большим значением среднего балла присвойте меньшее значение ранга. Значение среднего балла округлите до двух десятичных знаков.

In [21]:
subq = select(func.avg(exam_marks.c.mark).label("avg_mark"))
subq = subq.where(exam_marks.c.subj_id == subject.c.subj_id).scalar_subquery()

stmt = select(subject.c.subj_name, func.round(subq, 2).label("avg_mark"), func.dense_rank().over(order_by=desc(subq)).label("rank"))
stmt = stmt.where(subq != None)
print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))

for row in result:
    print(f"rank: {row.rank}, subject: {row.subj_name}, avg_mark: {row.avg_mark}")
    

SELECT subject.subj_name, round((SELECT avg(exam_marks.mark) AS avg_mark 
FROM exam_marks 
WHERE exam_marks.subj_id = subject.subj_id), :round_1) AS avg_mark, dense_rank() OVER (ORDER BY (SELECT avg(exam_marks.mark) AS avg_mark 
FROM exam_marks 
WHERE exam_marks.subj_id = subject.subj_id) DESC) AS rank 
FROM subject 
WHERE (SELECT avg(exam_marks.mark) AS avg_mark 
FROM exam_marks 
WHERE exam_marks.subj_id = subject.subj_id) IS NOT NULL
10
rank: 1, subject: Обработка данных и моделирование в Excel, avg_mark: 4.0
rank: 2, subject: Английский язык, avg_mark: 3.75
rank: 3, subject: Информатика, avg_mark: 3.33
rank: 3, subject: Программирование на языках Python и SQL, avg_mark: 3.33
rank: 4, subject: История, avg_mark: 3.14
rank: 5, subject: Физкультура, avg_mark: 3.0
rank: 6, subject: Физика, avg_mark: 2.83
rank: 7, subject: ОБЖ, avg_mark: 2.5
rank: 8, subject: Анализ данных, avg_mark: 2.0
rank: 8, subject: Математика, avg_mark: 2.0


## 3. Ранжируйте преподавателей (без пропуска значений ранга) по количеству преподаваемых ими предметов.

In [29]:
subq = select(func.count(subj_lect.c.subj_id).label("subj_amt"))
subq = subq.where(subj_lect.c.lecturer_id == lecturer.c.lecturer_id).scalar_subquery()

stmt = select(lecturer.c.surname, lecturer.c.name, subq.label("subj_amt"), func.dense_rank().over(order_by=desc(subq)).label("rank"))
stmt = stmt.where(subq != None)
print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))

for row in result:
    print(f"rank: {row.rank}, lecturer: {row.surname + ' ' + row.name}, subj_amt: {row.subj_amt}")

    

SELECT lecturer.surname, lecturer.name, (SELECT count(subj_lect.subj_id) AS subj_amt 
FROM subj_lect 
WHERE subj_lect.lecturer_id = lecturer.lecturer_id) AS subj_amt, dense_rank() OVER (ORDER BY (SELECT count(subj_lect.subj_id) AS subj_amt 
FROM subj_lect 
WHERE subj_lect.lecturer_id = lecturer.lecturer_id) DESC) AS rank 
FROM lecturer 
WHERE (SELECT count(subj_lect.subj_id) AS subj_amt 
FROM subj_lect 
WHERE subj_lect.lecturer_id = lecturer.lecturer_id) IS NOT NULL
29
rank: 1, lecturer: Зайцева Юлия, subj_amt: 2
rank: 1, lecturer: Максимова Иванна, subj_amt: 2
rank: 1, lecturer: Куликова Ираида, subj_amt: 2
rank: 1, lecturer: Волков Михаил, subj_amt: 2
rank: 1, lecturer: Беспалов Юлиан, subj_amt: 2
rank: 1, lecturer: Струков Николай, subj_amt: 2
rank: 2, lecturer: Сафонова Лукия, subj_amt: 1
rank: 2, lecturer: Мамонтова Акулина, subj_amt: 1
rank: 2, lecturer: Брагина Нинель, subj_amt: 1
rank: 2, lecturer: Кудрявцева Анна, subj_amt: 1
rank: 2, lecturer: Дорофеев Прокофий, subj_amt: 1
r

## 4. Ранжируйте семестры по количеству учебных часов.

In [13]:
sj1 = subject.alias("sj1")
sj2 = subject.alias("sj2")

subq = select(func.sum(sj1.c.hour).label("ttl_hour"))
subq = subq.where(sj1.c.semester == sj2.c.semester).scalar_subquery()

stmt = select(sj2.c.semester.distinct(), subq.label("ttl_hour"), func.dense_rank().over(order_by=desc(subq)).label("rank"))
stmt = stmt.where(subq != None)
print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))

for row in result:
    print(f"rank: {row.rank}, semester: {row.semester}, total_hours: {row.ttl_hour}")


    

SELECT DISTINCT sj2.semester, (SELECT sum(sj1.hour) AS ttl_hour 
FROM subject AS sj1 
WHERE sj1.semester = sj2.semester) AS ttl_hour, dense_rank() OVER (ORDER BY (SELECT sum(sj1.hour) AS ttl_hour 
FROM subject AS sj1 
WHERE sj1.semester = sj2.semester) DESC) AS rank 
FROM subject AS sj2 
WHERE (SELECT sum(sj1.hour) AS ttl_hour 
FROM subject AS sj1 
WHERE sj1.semester = sj2.semester) IS NOT NULL
5
rank: 1, semester: 4, total_hours: 138
rank: 2, semester: 1, total_hours: 132
rank: 3, semester: 2, total_hours: 124
rank: 4, semester: 3, total_hours: 92
rank: 5, semester: 5, total_hours: 34


## 5. Рассчитайте относительный прирост суммарного количества учебных часов предметов семестра по мере увеличения номера семестра.

In [19]:
sj1 = subject.alias("sj1")
sj2 = subject.alias("sj2")

subq = select(func.sum(sj1.c.hour).label("ttl_hour"))
subq = subq.where(sj1.c.semester == sj2.c.semester).scalar_subquery()

stmt_subq = select(sj2.c.semester.distinct().label("semester"), subq.label("ttl_hour"))
stmt_subq = stmt_subq.where(subq != None).subquery()


stmt = select(stmt_subq.c.semester, stmt_subq.c.ttl_hour, func.round(((stmt_subq.c.ttl_hour*1.0 / func.lag(stmt_subq.c.ttl_hour).over(order_by = stmt_subq.c.semester) - 1)*100.0), 2).label("diff"))
stmt = stmt.select_from(stmt_subq)

print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))

for row in result:
    print(f"semester: {row.semester}, total_hours: {row.ttl_hour},  growth: {row.diff} %")


SELECT anon_1.semester, anon_1.ttl_hour, round(((anon_1.ttl_hour * :ttl_hour_1) / lag(anon_1.ttl_hour) OVER (ORDER BY anon_1.semester) - :param_1) * :param_2, :round_1) AS diff 
FROM (SELECT DISTINCT sj2.semester AS semester, (SELECT sum(sj1.hour) AS ttl_hour 
FROM subject AS sj1 
WHERE sj1.semester = sj2.semester) AS ttl_hour 
FROM subject AS sj2 
WHERE (SELECT sum(sj1.hour) AS ttl_hour 
FROM subject AS sj1 
WHERE sj1.semester = sj2.semester) IS NOT NULL) AS anon_1
5
semester: 1, total_hours: 132,  growth: None %
semester: 2, total_hours: 124,  growth: -6.06 %
semester: 3, total_hours: 92,  growth: -25.81 %
semester: 4, total_hours: 138,  growth: 50.0 %
semester: 5, total_hours: 34,  growth: -75.36 %


## 6. Для каждого студента университета, обучающегося в своём городе, рассчитайте разницу в возрасте в днях с самым старшим студентом своего университета.

### Решение с оконными функциями

In [8]:
stmt = select(student.c.surname, student.c.name, student.c.city.label("std_city"), university.c.city.label("univ_city"), func.julianday(func.date(student.c.birthday)) - func.julianday(func.min(func.date(student.c.birthday)).over(partition_by=student.c.univ_id)))
stmt = stmt.select_from(student)
stmt = stmt.join(university, university.c.univ_id ==  student.c.univ_id).subquery()

stmt = select(stmt).where(stmt.c.std_city == stmt.c.univ_city)
print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))
result

SELECT anon_1.surname, anon_1.name, anon_1.std_city, anon_1.univ_city, anon_1.anon_2 
FROM (SELECT student.surname AS surname, student.name AS name, student.city AS std_city, university.city AS univ_city, julianday(date(student.birthday)) - julianday(min(date(student.birthday)) OVER (PARTITION BY student.univ_id)) AS anon_2 
FROM student JOIN university ON university.univ_id = student.univ_id) AS anon_1 
WHERE anon_1.std_city = anon_1.univ_city
9


[('Белкин', 'Вадим', 'Воронеж', 'Воронеж', 1898.0),
 ('Павлов', 'Андрей', 'Воронеж', 'Воронеж', 2201.0),
 ('Лукин', 'Артем', 'Воронеж', 'Воронеж', 2592.0),
 ('Журавлева', 'Вера', 'Новосибирск', 'Новосибирск', 3004.0),
 ('Котов', 'Павел', 'Белгород', 'Белгород', 2055.0),
 ('Петрова', 'Екатерина', 'Белгород', 'Белгород', 2137.0),
 ('Сидоров', 'Вадим', 'Москва', 'Москва', 1103.0),
 ('Баранов', 'Александр', 'Москва', 'Москва', 808.0),
 ('Миронова', 'Федор', 'Москва', 'Москва', 2602.0)]

### Решение без оконных функций 

In [7]:
s1 = student.alias("s1")
s2 = student.alias("s2")

subq = select(func.date(func.min(s2.c.birthday)))
subq = subq.group_by(s2.c.univ_id)
subq = subq.having(s2.c.univ_id == university.c.univ_id).scalar_subquery()


stmt = select(s1.c.surname, s1.c.name, university.c.univ_name, func.julianday(func.date(s1.c.birthday)) - func.julianday(subq))
stmt = stmt.select_from(s1)
stmt = stmt.join(university, university.c.univ_id == s1.c.univ_id)
stmt = stmt.where(s1.c.city == university.c.city)


print(stmt)
result = conn.execute(stmt).fetchall()
print(len(result))

result[:10]


SELECT s1.surname, s1.name, university.univ_name, julianday(date(s1.birthday)) - julianday((SELECT date(min(s2.birthday)) AS date_1 
FROM student AS s2 GROUP BY s2.univ_id 
HAVING s2.univ_id = university.univ_id)) AS anon_1 
FROM student AS s1 JOIN university ON university.univ_id = s1.univ_id 
WHERE s1.city = university.city
9


[('Сидоров', 'Вадим', 'МГУ', 1103.0),
 ('Котов', 'Павел', 'БГУ', 2055.0),
 ('Белкин', 'Вадим', 'ВГУ', 1898.0),
 ('Журавлева', 'Вера', 'НГУ', 3004.0),
 ('Баранов', 'Александр', 'ФинУ', 808.0),
 ('Петрова', 'Екатерина', 'БГУ', 2137.0),
 ('Миронова', 'Федор', 'МГТУ им. Баумана', 2602.0),
 ('Павлов', 'Андрей', 'ВГУ', 2201.0),
 ('Лукин', 'Артем', 'ВГУ', 2592.0)]